In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import os

In [2]:
# Loading Data

output_dir = "outputs"

csv_path = os.path.join(output_dir, "OD_ADULTS_Timeindex_Motive_location_tripexpansionfactors.csv")
final_df = pd.read_csv(csv_path, header=[0,1], low_memory=False)

gdf_path = os.path.join(output_dir, "H3_res9/Municipios_2023_h3_res9.shp")
hex_gdf = gpd.read_file(gdf_path)
if 'h3_index' in hex_gdf.columns:
    hex_gdf = hex_gdf.rename(columns={'h3_index': 'hexagon'})
hex_gdf['hexagon'] = hex_gdf['hexagon'].astype(str)


hex_df = final_df.xs('hexagon', level=1, axis=1)
motive_df = final_df.xs('motive', level=1, axis=1)
expf_df = final_df.xs('tripexpfactor', level=1, axis=1)

urbanised_gdf = gpd.read_file("ucl-mres-thesis/SPMA Urbanised Areas-MapBiomas/2022/AU_2022.shp")


In [3]:
# CRS alignment for all spatial joins/plots
if hex_gdf.crs != urbanised_gdf.crs:
    urbanised_gdf = urbanised_gdf.to_crs(hex_gdf.crs)
# Urbanised hexagons: spatial join
urban_hex_gdf = gpd.sjoin(hex_gdf, urbanised_gdf, how='inner', predicate='intersects').drop(columns=['index_right'])
urban_hex_set = set(urban_hex_gdf['hexagon'])

In [4]:
hex_long = hex_df.stack().rename('hexagon')
motive_long = motive_df.stack().rename('motive')
expf_long = expf_df.stack().rename('tripexpfactor')
long_df = pd.concat([hex_long, motive_long, expf_long], axis=1).reset_index()
long_df = long_df[(long_df['hexagon'] != 'transit') & (long_df['motive'] != 'transit') & (long_df['tripexpfactor'] != 'transit')]
long_df['hexagon'] = long_df['hexagon'].astype(str)
long_df['motive'] = long_df['motive'].astype(int)
long_df['tripexpfactor'] = pd.to_numeric(long_df['tripexpfactor'], errors='coerce')

In [5]:
# Calculating entropy in urbanised area only
def entropy_weighted(counts):
    p = np.array(counts, dtype=float)
    total = p.sum()
    if total == 0:
        return 0
    p = p / total
    p = p[p > 0]
    return -np.sum(p * np.log2(p))

motive_codes = sorted(long_df['motive'].unique())
entropy_results = []
for h, group in long_df.groupby('hexagon'):
    motive_weights = group.groupby('motive')['tripexpfactor'].sum()
    motive_weights = motive_weights.reindex(motive_codes, fill_value=0)
    ent = entropy_weighted(motive_weights.values)
    entropy_results.append({'hexagon': h, 'motive_entropy': ent})
entropy_df = pd.DataFrame(entropy_results)

In [6]:
# Merging with urban hexes for plotting
urban_hex_gdf['hexagon'] = urban_hex_gdf['hexagon'].astype(str)
hex_entropy_map = urban_hex_gdf.merge(entropy_df, on="hexagon", how="left").fillna(0)

In [7]:
# Plotting 
fig, ax = plt.subplots(figsize=(14, 12))

urbanised_gdf.plot(ax=ax, facecolor='lightgray', edgecolor='gray', linewidth=1, zorder=0)
hex_entropy_map.plot(
    column='motive_entropy',
    cmap='hot',
    legend=True,
    legend_kwds={'shrink': 0.5},
    edgecolor="black",
    linewidth=0.1,
    ax=ax,
    zorder=1
)

hex_gdf.plot(ax=ax, facecolor='gainsboro', linewidth=1, zorder=0)

plt.title("Expansion-Weighted Motive Entropy per Hexagon (Urbanised Areas Only)", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.savefig(os.path.join(output_dir,"hexagon_motive_entropy_weighted_URBANISED.png"), dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close()


In [8]:
summary_stats = entropy_df['motive_entropy'].describe()
print(summary_stats)

count    11427.000000
mean         0.509073
std          0.589494
min         -0.000000
25%         -0.000000
50%          0.257723
75%          0.947145
max          2.729837
Name: motive_entropy, dtype: float64


In [9]:
# Create list of quarter-hour time labels
quarter_labels = [f"{h:02d}:{m:02d}" for h in range(24) for m in [0, 15, 30, 45]]

In [10]:
# Compute mean entropy per period (across urbanised hexes)
mean_entropy_list = []
for i in range(96):
    hex_col = hex_df.iloc[:, i]
    motive_col = motive_df.iloc[:, i]
    expf_col = expf_df.iloc[:, i]
    long_df = pd.DataFrame({
        'hexagon': hex_col,
        'motive': motive_col,
        'tripexpfactor': expf_col
    })
    long_df = long_df[
        (long_df['hexagon'] != 'transit') &
        (long_df['motive'] != 'transit') &
        (long_df['tripexpfactor'] != 'transit')
    ]
    long_df['hexagon'] = long_df['hexagon'].astype(str)
    long_df['motive'] = pd.to_numeric(long_df['motive'], errors='coerce')
    long_df['tripexpfactor'] = pd.to_numeric(long_df['tripexpfactor'], errors='coerce')
    long_df = long_df.dropna(subset=['motive', 'tripexpfactor'])
    long_df = long_df[long_df['hexagon'].isin(urban_hex_set)]
    entropy_vals = []
    motive_codes = sorted(long_df['motive'].unique())
    for hexagon, group in long_df.groupby('hexagon'):
        motive_weights = group.groupby('motive')['tripexpfactor'].sum()
        motive_weights = motive_weights.reindex(motive_codes, fill_value=0)
        entropy_vals.append(entropy_weighted(motive_weights.values))
    # Take the mean for this period (skip empty slices)
    if entropy_vals:
        mean_entropy_list.append(np.mean(entropy_vals))
    else:
        mean_entropy_list.append(np.nan)

# Plot the results
plt.figure(figsize=(16, 6))
plt.plot(quarter_labels, mean_entropy_list, marker='o', color='crimson')
plt.xticks(range(0, 96, 4), [quarter_labels[i] for i in range(0, 96, 4)], rotation=45, ha='right')
plt.xlabel("Quarter-hour interval")
plt.ylabel("Mean Motive Entropy\n(per Urbanised Hexagon)")
plt.title("Temporal Profile of Mean Motive Entropy across Urbanised Area (15-Minute Resolution)")
plt.grid(True, alpha=0.4)
plt.tight_layout()
plt.savefig(os.path.join(output_dir,"urbanised_mean_motive_entropy_quarter_hour.png"), dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close()


In [11]:
# ========== LISA AND PEAK-WINDOW ENTROPY ==========

from matplotlib.colors import ListedColormap

peak_windows = {
    '06:00–08:00': list(range(24, 32)),
    '12:00–14:00': list(range(48, 56)),
    '17:00–19:00': list(range(68, 76)),
}


In [12]:
# Reuse this from earlier, define if not already:
def clean_long_df(long_df, urban_hex_set, drop_motives=None):
    mask = (
        (long_df['hexagon'] != 'transit') &
        (long_df['motive'] != 'transit') &
        (long_df['tripexpfactor'] != 'transit')
    )
    dfc = long_df.loc[mask].copy()
    dfc['hexagon'] = dfc['hexagon'].astype(str)
    dfc['motive'] = pd.to_numeric(dfc['motive'], errors='coerce')
    dfc['tripexpfactor'] = pd.to_numeric(dfc['tripexpfactor'], errors='coerce')
    dfc = dfc.dropna(subset=['motive', 'tripexpfactor'])
    dfc = dfc[dfc['hexagon'].isin(urban_hex_set)]
    if drop_motives is not None:
        dfc = dfc[~dfc['motive'].isin(drop_motives)]
    return dfc


In [13]:
from libpysal.weights import Queen
from esda.moran import Moran, Moran_Local

# Custom colormap and cluster labels
from matplotlib.colors import ListedColormap
lisa_colormap = ListedColormap([
    '#000000','#ff0000','#0000ff','#ffff00','#00ff00'
])
cluster_labels = ['Non-significant', 'High-High', 'Low-Low', 'Low-High', 'High-Low']

entropy_map_results = {}

for label, idxs in peak_windows.items():
    # Get the relevant columns for each time window
    hex_win = hex_df.iloc[:, idxs]
    motive_win = motive_df.iloc[:, idxs]
    expf_win = expf_df.iloc[:, idxs]
    # Stack into long format
    hex_long = hex_win.stack().rename('hexagon')
    motive_long = motive_win.stack().rename('motive')
    expf_long = expf_win.stack().rename('tripexpfactor')
    long_df_p = pd.concat([hex_long, motive_long, expf_long], axis=1).reset_index(drop=True)
    clean_df = clean_long_df(long_df_p, urban_hex_set)

    motive_codes = sorted(clean_df['motive'].unique())
    entropy_list = []
    for h, group in clean_df.groupby('hexagon'):
        motive_weights = group.groupby('motive')['tripexpfactor'].sum()
        motive_weights = motive_weights.reindex(motive_codes, fill_value=0)
        ent = entropy_weighted(motive_weights.values)
        entropy_list.append({'hexagon': h, 'motive_entropy': ent})
    entropy_df_p = pd.DataFrame(entropy_list)

    # Merge with spatial info
    map_gdf = urban_hex_gdf.merge(entropy_df_p, on='hexagon', how='left').fillna({'motive_entropy': 0})
    entropy_map_results[label] = map_gdf


In [14]:
w = Queen.from_dataframe(urban_hex_gdf, use_index=False)


/opt/conda/lib/python3.11/site-packages/libpysal/weights/contiguity.py:347: UserWarning: The weights matrix is not fully connected: 
 There are 282 disconnected components.
 There are 51 islands with ids: 618, 974, 1016, 1513, 2091, 2297, 3127, 4152, 4639, 5031, 5101, 5484, 5535, 5746, 6367, 6369, 6500, 7176, 7879, 8229, 8401, 9263, 9964, 10437, 11057, 11277, 11669, 12368, 13403, 13828, 14623, 14884, 15413, 17010, 17055, 17188, 17208, 17378, 17681, 19128, 19418, 19722, 19828, 19924, 20314, 21028, 21101, 21281, 21822, 22217, 22234.
  W.__init__(self, neighbors, ids=ids, **kw)


In [15]:
for label, gdf in entropy_map_results.items():
    y = gdf['motive_entropy'].values
    moran = Moran(y, w)
    print(f"{label}: Moran's I = {moran.I:.3f}, p = {moran.p_sim:.3f}")
    lisa = Moran_Local(y, w)
    gdf['lisa_q'] = lisa.q
    gdf['lisa_p'] = lisa.p_sim
    sig = gdf['lisa_p'] < 0.05
    cluster = gdf['lisa_q']
    gdf['lisa_type'] = np.where(sig, cluster, 0)
    gdf['lisa_type'] = pd.to_numeric(gdf['lisa_type'], downcast='integer', errors='coerce').fillna(0).astype(int)
    # Save back into the dict for possible export
    entropy_map_results[label] = gdf


('WARNING: ', 618, ' is an island (no neighbors)')
('WARNING: ', 974, ' is an island (no neighbors)')
('WARNING: ', 1016, ' is an island (no neighbors)')
('WARNING: ', 1513, ' is an island (no neighbors)')
('WARNING: ', 2091, ' is an island (no neighbors)')
('WARNING: ', 2297, ' is an island (no neighbors)')
('WARNING: ', 3127, ' is an island (no neighbors)')
('WARNING: ', 4152, ' is an island (no neighbors)')
('WARNING: ', 4639, ' is an island (no neighbors)')
('WARNING: ', 5031, ' is an island (no neighbors)')
('WARNING: ', 5101, ' is an island (no neighbors)')
('WARNING: ', 5484, ' is an island (no neighbors)')
('WARNING: ', 5535, ' is an island (no neighbors)')
('WARNING: ', 5746, ' is an island (no neighbors)')
('WARNING: ', 6367, ' is an island (no neighbors)')
('WARNING: ', 6369, ' is an island (no neighbors)')
('WARNING: ', 6500, ' is an island (no neighbors)')
('WARNING: ', 7176, ' is an island (no neighbors)')
('WARNING: ', 7879, ' is an island (no neighbors)')
('WARNING: ', 

/opt/conda/lib/python3.11/site-packages/esda/moran.py:1084: RuntimeWarning: invalid value encountered in divide
  self.z_sim = (self.Is - self.EI_sim) / self.seI_sim


12:00–14:00: Moran's I = 0.433, p = 0.001


/opt/conda/lib/python3.11/site-packages/esda/moran.py:1084: RuntimeWarning: invalid value encountered in divide
  self.z_sim = (self.Is - self.EI_sim) / self.seI_sim


17:00–19:00: Moran's I = 0.408, p = 0.001


/opt/conda/lib/python3.11/site-packages/esda/moran.py:1084: RuntimeWarning: invalid value encountered in divide
  self.z_sim = (self.Is - self.EI_sim) / self.seI_sim


In [16]:
for label, gdf in entropy_map_results.items():
    fig, ax = plt.subplots(figsize=(13,12))
    urbanised_gdf.plot(ax=ax, facecolor='lightgray', edgecolor='gray', linewidth=0.3, zorder=0)
    hex_gdf.boundary.plot(ax=ax, facecolor='gainsboro', color='white', linewidth=0.1, zorder=2)
    gdf.plot(column='lisa_type', cmap=lisa_colormap, ax=ax, legend=False,
             edgecolor='k', linewidth=0.1, vmin=0, vmax=4, zorder=3)
    for i, l in enumerate(cluster_labels):
        ax.scatter([], [], color=lisa_colormap.colors[i], label=l)
    ax.legend(frameon=True, fontsize=13, title="LISA Cluster", loc="lower left")
    ax.set_title(f"LISA Clusters Motive Entropy ({label}), Urbanised Hexagons", fontsize=17)
    ax.axis('off')
    plt.tight_layout()
    fname = f"lisa_clusters_motive_entropy_urbanised_{label.replace(':','').replace('–','-')}.png"
    plt.savefig(os.path.join(output_dir, fname), dpi=300, bbox_inches='tight')
    plt.close()

In [17]:
# Build counts per window in a single table
summary = pd.DataFrame()
for label, gdf in entropy_map_results.items():
    counts = gdf['lisa_type'].value_counts().reindex(range(5), fill_value=0)
    summary = pd.concat([summary, counts.rename(label)], axis=1)
summary = summary.T
summary.columns = cluster_labels
summary_path = os.path.join(output_dir, "MOTIVE_Lisa_Cluster_Summary.csv")
summary.to_csv(summary_path, index=True)
# Exclude Non-significant column for the plot
summary_no_ns = summary.drop(columns=['Non-significant'])
bar_colors_no_ns = lisa_colormap.colors[1:]  # HH, LL, LH, HL

fig, ax = plt.subplots(figsize=(10, 7))
bottom = np.zeros(len(summary_no_ns))
for col, color in zip(summary_no_ns.columns, bar_colors_no_ns):
    ax.bar(summary_no_ns.index, summary_no_ns[col].values, label=col, bottom=bottom, color=color)
    bottom += summary_no_ns[col].values

ax.set_ylabel('Number of hexagons')
ax.set_title('LISA Cluster Types by Time Period (Excluding Non-significant)\n(Urbanised Area only)')
ax.legend(title='LISA Cluster Type')
plt.xticks(rotation=0)
plt.tight_layout()
plot_path = os.path.join(output_dir, 'MOTIVE_Lisa_Cluster_Types_by_Time_Period_Urbanised_NoNonSig.png')
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
plt.close()


In [18]:
print(summary)

             Non-significant  High-High  Low-Low  Low-High  High-Low
06:00–08:00            19275       1493     1004       133       378
12:00–14:00            13162       2125      872      5837       287
17:00–19:00            19289       1678      879       126       311


In [19]:
map_gdf.to_csv(os.path.join(output_dir,"motive_entropy.csv"), index=False)


In [20]:
# Entropy of Economic groups

In [21]:
def econ_group(code):
    code = int(code)
    if code == 1: return 'High (A)'
    elif code in [2,3]: return 'Upper middle (B1/B2)'
    elif code in [4,5]: return 'Lower middle (C1/C2)'
    elif code == 6: return 'Low (D/E)'
    return 'Unknown'

econ_groups = ['High (A)', 'Upper middle (B1/B2)', 'Lower middle (C1/C2)', 'Low (D/E)']

In [22]:
cleaned_path = os.path.join(output_dir, "cleandbf.csv")
cleaned_df = pd.read_csv(cleaned_path, dtype=str, low_memory=False)
cleaned_df['econ_group'] = cleaned_df['CRITERIOBR'].apply(econ_group)
person_exp_df = cleaned_df.drop_duplicates(subset='ID_PESS')[['ID_PESS','FE_PESS','econ_group']]
person_exp_df['FE_PESS'] = person_exp_df['FE_PESS'].astype(float)
person_exp = person_exp_df.set_index('ID_PESS').to_dict(orient='index')

In [23]:
person_hex_path = os.path.join(output_dir, "OD_ADULTS_Timeindex_Motive_location_tripexpansionfactors.csv")
person_hex_df = pd.read_csv(person_hex_path, dtype=str, low_memory=False)

# Whole day: all intervals as strings
all_cols = list(map(str, range(96)))
wholeday_long = person_hex_df.melt(id_vars=['ID_PESS'],
                                   value_vars=all_cols,
                                   var_name='interval',
                                   value_name='hexagon')  # wide→long
wholeday_long = wholeday_long[wholeday_long['hexagon'] != 'transit']
wholeday_long['hexagon'] = wholeday_long['hexagon'].astype(str)
wholeday_long['FE_PESS'] = wholeday_long['ID_PESS'].map(lambda x: person_exp.get(x, {}).get('FE_PESS', np.nan))
wholeday_long['econ_group'] = wholeday_long['ID_PESS'].map(lambda x: person_exp.get(x, {}).get('econ_group', 'Unknown'))
wholeday_long = wholeday_long.dropna(subset=['FE_PESS'])
# Urbanised only
wholeday_long = wholeday_long[wholeday_long['hexagon'].isin(urban_hex_set)]

# Entropy per hex
entropy_results = []
for hx, grp in wholeday_long.groupby('hexagon'):
    s = grp.groupby('econ_group')['FE_PESS'].sum().reindex(econ_groups, fill_value=0)
    entropy_results.append({'hexagon': hx, 'econ_entropy': entropy_weighted(s.values)})
econ_entropy_df = pd.DataFrame(entropy_results)

# Joining to geo and plotting
econ_entropy_map = urban_hex_gdf.merge(econ_entropy_df, on="hexagon", how="left").fillna(0)

fig, ax = plt.subplots(figsize=(14, 12))
urbanised_gdf.plot(ax=ax, facecolor='lightgray', edgecolor='gray', linewidth=1, zorder=0)
econ_entropy_map.plot(column='econ_entropy', cmap='hot', legend=True,
                      legend_kwds={'shrink': 0.5}, edgecolor="black", linewidth=0.1, ax=ax, zorder=1)
hex_gdf.plot(ax=ax, facecolor='gainsboro', linewidth=1, zorder=0)
plt.title("Expansion-Weighted Economic Group Entropy per Hexagon (Whole Day, Urbanised Areas Only)", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.savefig(os.path.join(output_dir,"hexagon_econ_entropy_weighted_URBANISED.png"),
            dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close()

print(econ_entropy_df['econ_entropy'].describe())


count    11147.000000
mean         0.521250
std          0.526707
min         -0.000000
25%         -0.000000
50%          0.467868
75%          0.970417
max          1.979707
Name: econ_entropy, dtype: float64


In [24]:
quarter_labels = [f"{h:02d}:{m:02d}" for h in range(24) for m in [0, 15, 30, 45]]

mean_econ_entropy = []
for i in range(96):
    win_long = person_hex_df.melt(id_vars=['ID_PESS'], value_vars=[str(i)],
                                  var_name='interval', value_name='hexagon')
    win_long = win_long[win_long['hexagon'] != 'transit']
    win_long['hexagon'] = win_long['hexagon'].astype(str)
    win_long['FE_PESS'] = win_long['ID_PESS'].map(lambda x: person_exp.get(x, {}).get('FE_PESS', np.nan))
    win_long['econ_group'] = win_long['ID_PESS'].map(lambda x: person_exp.get(x, {}).get('econ_group', 'Unknown'))
    win_long = win_long.dropna(subset=['FE_PESS'])
    win_long = win_long[win_long['hexagon'].isin(urban_hex_set)]
    ent_vals = []
    for h, g in win_long.groupby('hexagon'):
        weights = g.groupby('econ_group')['FE_PESS'].sum().reindex(econ_groups, fill_value=0)
        ent_vals.append(entropy_weighted(weights.values))
    mean_econ_entropy.append(np.mean(ent_vals) if ent_vals else np.nan)

plt.figure(figsize=(16, 6))
plt.plot(quarter_labels, mean_econ_entropy, marker='o', color='navy')
plt.xticks(range(0, 96, 4), [quarter_labels[i] for i in range(0, 96, 4)], rotation=45, ha='right')
plt.xlabel("Quarter-hour interval"); plt.ylabel("Mean Economic Group Entropy\n(per Urbanised Hexagon)")
plt.title("Temporal Profile of Mean Economic Group Entropy across Urbanised Area (15-Minute Resolution)")
plt.grid(True, alpha=0.4); plt.tight_layout()
plt.savefig(os.path.join(output_dir, "urbanised_mean_econgroup_entropy_quarter_hour.png"),
            dpi=300, bbox_inches='tight', pad_inches=0.1)
plt.close()


In [25]:
peak_windows_str = {k: list(map(str, v)) for k, v in peak_windows.items()}

econ_entropy_maps = {}

# Computing entropy per peak window
for window_label, cols in peak_windows_str.items():
    win_long = person_hex_df.melt(id_vars=['ID_PESS'], value_vars=cols,
                                  var_name='interval', value_name='hexagon')
    win_long = win_long[win_long['hexagon'] != 'transit']
    win_long['hexagon'] = win_long['hexagon'].astype(str)
    win_long['FE_PESS'] = win_long['ID_PESS'].map(lambda x: person_exp.get(x, {}).get('FE_PESS', np.nan))
    win_long['econ_group'] = win_long['ID_PESS'].map(lambda x: person_exp.get(x, {}).get('econ_group', 'Unknown'))
    win_long = win_long.dropna(subset=['FE_PESS'])
    # Urban filter
    win_long = win_long[win_long['hexagon'].isin(urban_hex_set)]
    
    # Entropy per hex
    ent_rows = []
    for hx, grp in win_long.groupby('hexagon'):
        s = grp.groupby('econ_group')['FE_PESS'].sum().reindex(econ_groups, fill_value=0)
        ent_rows.append({'hexagon': hx, 'econ_entropy': entropy_weighted(s.values)})
    ent_df = pd.DataFrame(ent_rows)

    # Join to geometry; store for LISA
    gdf_win = urban_hex_gdf.merge(ent_df, on='hexagon', how='left').fillna({'econ_entropy': 0})
    econ_entropy_maps[window_label] = gdf_win

In [26]:
# LISA per peak window using the same spatial weights w
from esda.moran import Moran, Moran_Local

for label, gdf_win in econ_entropy_maps.items():
    y = gdf_win['econ_entropy'].values
    moran = Moran(y, w)
    print(f"{label}: Moran's I = {moran.I:.3f}, p = {moran.p_sim:.3f}")
    lisa = Moran_Local(y, w)
    sig = lisa.p_sim < 0.05
    gdf_win['lisa_type'] = np.where(sig, lisa.q, 0).astype(int)

    # Plot LISA clusters (exclude separate non-sig plot as requested)
    fig, ax = plt.subplots(figsize=(13,12))
    urbanised_gdf.plot(ax=ax, facecolor='lightgray', edgecolor='gray', linewidth=0.3, zorder=0)
    hex_gdf.boundary.plot(ax=ax, facecolor='gainsboro', color='white', linewidth=0.1, zorder=2)
    gdf_win.plot(column='lisa_type', cmap=lisa_colormap, ax=ax, legend=False,
                 edgecolor='k', linewidth=0.1, vmin=0, vmax=4, zorder=3)
    for i, l in enumerate(cluster_labels):
        ax.scatter([], [], color=lisa_colormap.colors[i], label=l)
    ax.legend(frameon=True, fontsize=13, title="LISA Cluster", loc="lower left")
    ax.set_title(f"LISA Clusters of Economic Entropy ({label})\nUrbanised Hexagons, p < 0.05", fontsize=17)
    ax.axis('off')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"lisa_clusters_econ_entropy_urbanised_{label.replace(':','').replace('–','-')}.png"),
                dpi=300, bbox_inches='tight')
    plt.close()

06:00–08:00: Moran's I = 0.388, p = 0.001


/opt/conda/lib/python3.11/site-packages/esda/moran.py:1084: RuntimeWarning: invalid value encountered in divide
  self.z_sim = (self.Is - self.EI_sim) / self.seI_sim


12:00–14:00: Moran's I = 0.433, p = 0.001


/opt/conda/lib/python3.11/site-packages/esda/moran.py:1084: RuntimeWarning: invalid value encountered in divide
  self.z_sim = (self.Is - self.EI_sim) / self.seI_sim


17:00–19:00: Moran's I = 0.396, p = 0.001


/opt/conda/lib/python3.11/site-packages/esda/moran.py:1084: RuntimeWarning: invalid value encountered in divide
  self.z_sim = (self.Is - self.EI_sim) / self.seI_sim


In [27]:
# Summary counts including Non-significant
econ_summary = pd.DataFrame()
for label, gdf_e in econ_entropy_maps.items():
    counts = gdf_e['lisa_type'].value_counts().reindex(range(5), fill_value=0)
    econ_summary = pd.concat([econ_summary, counts.rename(label)], axis=1)
econ_summary = econ_summary.T
econ_summary.columns = cluster_labels
econ_summary.to_csv(os.path.join(output_dir, "ECON_Lisa_Cluster_Summary.csv"))

# Excluding Non-significant for the bar plot
econ_summary_no_ns = econ_summary.drop(columns=['Non-significant'])
bar_colors_no_ns = lisa_colormap.colors[1:]

fig, ax = plt.subplots(figsize=(10,7))
bottom = np.zeros(len(econ_summary_no_ns))
for col, color in zip(econ_summary_no_ns.columns, bar_colors_no_ns):
    ax.bar(econ_summary_no_ns.index, econ_summary_no_ns[col].values, label=col, bottom=bottom, color=color)
    bottom += econ_summary_no_ns[col].values

ax.set_ylabel('Number of hexagons')
ax.set_title('LISA Cluster Types by Time Period (Excluding Non-significant)\nEconomic Group Entropy (Urbanised Area only)')
ax.legend(title='LISA Cluster Type')
plt.xticks(rotation=0); plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'ECONGROUP_Lisa_Cluster_Types_by_Time_Period_Urbanised_NoNonSig.png'),
            dpi=300, bbox_inches='tight')
plt.close()


In [28]:
gdf_win.to_csv(os.path.join(output_dir,"econ_entropy.csv"), index=False)

In [29]:
# Matching High–High LISA

municipal_gdf = gpd.read_file("ucl-mres-thesis/Municipal boundaries-ShapeFiles/Municipios_2023.shp")

if hex_gdf.crs != municipal_gdf.crs:
    municipal_gdf = municipal_gdf.to_crs(hex_gdf.crs)
    
from sklearn.metrics import cohen_kappa_score

#  Making lightweight dicts with only the needed columns and consistent names
motive_lisa = {}
econ_lisa = {}

for period, gdf in entropy_map_results.items():  # motive
    cols = [c for c in ['hexagon','geometry','lisa_type'] if c in gdf.columns]
    motive_lisa[period] = gdf[cols].copy()

for period, gdf in econ_entropy_maps.items():  # econ
    cols = [c for c in ['hexagon','geometry','lisa_type'] if c in gdf.columns]
    econ_lisa[period] = gdf[cols].copy()

# Ensuring period keys match between the two analyses 
common_periods = [p for p in motive_lisa.keys() if p in econ_lisa.keys()]
if not common_periods:
    raise RuntimeError("No overlapping periods between motive and econ LISA results.")

# Overlapping stats and HH-only matches per period
overlap_stats = {}
hh_matches_gdfs = {} 

for period in common_periods:
    g_m = motive_lisa[period].rename(columns={'lisa_type':'motive_lisa'})
    g_e = econ_lisa[period].rename(columns={'lisa_type':'econ_lisa'})

    
    merged = g_m.merge(g_e[['hexagon','econ_lisa']], on='hexagon', how='inner')
    cross = pd.crosstab(merged['motive_lisa'], merged['econ_lisa'])
    cross = cross.reindex(index=range(5), columns=range(5), fill_value=0)

    # Agreement metrics 
    agree = (merged['motive_lisa'] == merged['econ_lisa']).sum()
    total = len(merged)
    percent = 100 * agree / total if total > 0 else np.nan
    kappa = cohen_kappa_score(merged['motive_lisa'], merged['econ_lisa'])

    # HH-only matches
    hh_match = merged[(merged['motive_lisa'] == 1) & (merged['econ_lisa'] == 1)].copy()
    hh_matches_gdfs[period] = hh_match

    overlap_stats[period] = {
        'crosstab': cross,
        'agree': agree,
        'total': total,
        'percent': percent,
        'kappa': kappa,
        'hh_count': len(hh_match)
    }

# Printing summary to console and export CSVs
for period, stats in overlap_stats.items():
    print(f"\n=== Overlap Summary: {period} ===")
    print(stats['crosstab'])
    print(f"Exact code match: {stats['agree']}/{stats['total']} ({stats['percent']:.1f}%)")
    print(f"Cohen's kappa: {stats['kappa']:.3f}")
    print(f"HH matches: {stats['hh_count']}")

    safe = period.replace(':','').replace('–','-')
    stats['crosstab'].to_csv(os.path.join(output_dir, f"Overlap_Crosstab_{safe}.csv"))
    pd.DataFrame([{
        'period': period,
        'agree': stats['agree'],
        'total': stats['total'],
        'percent': stats['percent'],
        'kappa': stats['kappa'],
        'hh_count': stats['hh_count']
    }]).to_csv(os.path.join(output_dir, f"Overlap_Summary_{safe}.csv"), index=False)

import matplotlib.pyplot as plt
from matplotlib.patches import Patch


def plot_hh_matches(period, color='mediumvioletred'):
    g_m = motive_lisa[period].rename(columns={'lisa_type': 'motive_lisa'})
    g_e = econ_lisa[period].rename(columns={'lisa_type': 'econ_lisa'})
    merged = g_m.merge(g_e[['hexagon', 'econ_lisa']], on='hexagon', how='inner')
    gdf_hh = merged[(merged['motive_lisa'] == 1) & (merged['econ_lisa'] == 1)].copy()

    if gdf_hh.empty:
        print(f"No matching High–High clusters for {period}.")
        return

    fig, ax = plt.subplots(figsize=(13, 12))

    urbanised_gdf.plot(ax=ax, facecolor='lightgray', edgecolor='gray', linewidth=0.3, zorder=0)
    municipal_gdf.boundary.plot(ax=ax, color='black', linewidth=0.7, zorder=1)
    gdf_hh.plot(ax=ax, color='mediumvioletred', edgecolor='white', linewidth=0.12, zorder=2)

    # Legend
    handles = [Patch(facecolor=color, edgecolor='white', label="Matching HH", alpha=0.85)]
    ax.legend(handles=handles, loc='upper right', fontsize=15, frameon=True, title='Cluster Type')

    ax.set_title(f"Matching High–High LISA Clusters (Motive & Econ)\n{period}", fontsize=17)
    ax.axis('off')

    # Set limits with a buffer so nothing is cropped
    minx, miny, maxx, maxy = urbanised_gdf.total_bounds
    buffer = 0.1 * max(maxx - minx, maxy - miny)
    ax.set_xlim(minx - buffer, maxx + buffer)
    ax.set_ylim(miny - buffer, maxy + buffer)
    plt.tight_layout()
    safe = period.replace(':', '').replace('–', '-')
    plt.savefig(os.path.join(output_dir, f"Matching_HH_LISA_{safe}_doubleboundary.png"),
                dpi=300, bbox_inches='tight', pad_inches=0.1)
    plt.close()

for period in ['06:00–08:00','12:00–14:00','17:00–19:00']:
    if period in motive_lisa:
        plot_hh_matches(period)



=== Overlap Summary: 06:00–08:00 ===
econ_lisa        0     1    2     3    4
motive_lisa                             
0            16802   608  523  6819  209
1              289  1135   97     0    2
2              439   244  357     0    0
3                4     0    0   278    1
4              276     9    2     9   82
Exact code match: 18654/28185 (66.2%)
Cohen's kappa: 0.218
HH matches: 1135

=== Overlap Summary: 12:00–14:00 ===
econ_lisa        0     1    2     3    4
motive_lisa                             
0            12050   341  367  3265  205
1              347  1687  125     0    0
2              316   148  426     0    0
3             6640     0    6  1958   15
4              188     1    0    12   88
Exact code match: 16209/28185 (57.5%)
Cohen's kappa: 0.209
HH matches: 1687

=== Overlap Summary: 17:00–19:00 ===
econ_lisa        0     1    2     3    4
motive_lisa                             
0            20857   468  515  2942  241
1              294  1270  122     0  